In [17]:
import numpy as np
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [9]:
pip install pyttsx3

In [1]:
pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pyaudio

In [23]:
import pyaudio
import pyttsx3
import speech_recognition as sr


In [24]:
# checkpoint 
checkpoint = "microsoft/DialoGPT-medium"
# download and cache tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# download and cache pre-trained model
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [25]:
# Build a ChatBot class with all necessary modules to make a complete conversation
class ChatBot():
    # initialize
    def __init__(self):
        # once chat starts, the history will be stored for chat continuity
        self.chat_history_ids = None
        # make input ids global to use them anywhere within the object
        self.bot_input_ids = None
        # a flag to check whether to end the conversation
        self.end_chat = False
        # Initialize Text-to-Speech Engine
        self.engine = pyttsx3.init()
        # Initialize Speech Recognition Engine
        self.recognizer = sr.Recognizer()
        # initialize tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium",padding_side="left")
        self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        # set seed for consistency
        
        # greet while starting
        self.welcome()
        self.listen()


# Define a function to speak text
    def speak(self,text=""):
        self.engine.say(text)
        self.engine.runAndWait()
    
    def listen(self):
        with sr.Microphone() as source:
            audio = self.recognizer.listen(source)
        try:
            text = self.recognizer.recognize_google(audio)
            self.user_input(text)
            self.bot_response()
        except:
            self.bot_response("I'm sorry, I didn't catch that.")
        self.recognizer = sr.Recognizer()

    def welcome(self):
        time.sleep(2)
        self.speak("Lets begin the conversation ...")
        # some time to get user ready
        time.sleep(2)
        self.speak('Say "bye" or "quit" or "exit" to end chat \n')
        # give time to read what has been printed
        time.sleep(3)
        # Greet and introduce
        greeting = np.random.choice([
            "Welcome, I am ChatBot, here for your kind service",
            "Hey, Great day! I am your virtual assistant",
            "Hello, it's my pleasure meeting you",
        ])
        self.speak("ChatBot >>  " + greeting)
        
    def user_input(self, text=""):
        # end conversation if user wishes so
        if text.lower().strip() in ['bye', 'quit', 'exit']:
            # turn flag on 
            self.end_chat=True
            # a closing comment
            self.speak('ChatBot >>  See you soon! Bye!')
            time.sleep(1)
            self.speak('\nQuitting ChatBot ...')
        else:
            # continue chat, preprocess input text
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            self.new_user_input_ids = self.tokenizer.encode(text + self.tokenizer.eos_token, \
                                                       return_tensors='pt')
            
   
   

    def bot_response(self, text=None):
        # check if chat should end
        if self.end_chat:
            return
        
        # append the new user input tokens to the chat history
        # if chat has already begun
        if self.chat_history_ids is not None:
           self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1) 
        else:
            # if first entry, initialize bot_input_ids
            self.bot_input_ids = self.new_user_input_ids
            
        # define the new chat_history_ids based on the preceding chats
        # generated a response while limiting the total chat history to 1000 tokens, 
        self.chat_history_ids = self.model.generate(self.bot_input_ids,
                                                    max_length=1000,
                                                    pad_token_id=self.tokenizer.eos_token_id,
                                                    no_repeat_ngram_size=3,
                                                    do_sample=True,
                                                    top_k=100,
                                                    top_p=0.7,
                                                    temperature=0.8)
                                            
    
        # last ouput tokens from bot
        response = self.tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        # in case, bot fails to answer
        if response == "":
           response = self.random_response()
        # speak bot response
        self.speak("chatbot>>" +response)
        
        # If there is text to be spoken, speak it

        
    
    
    

In [26]:
# build a ChatBot object
bot = ChatBot()
# start chatting
while True:
    bot.speak()
   
    # receive user input
    bot.user_input()
    # check whether to end chat
    if bot.end_chat:
        break
    # output bot response
    bot.bot_response()    


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

KeyboardInterrupt: 